In [1]:
import pandas as pd
data = {
    'Model Variant': ['Without LoRA', 'QLoRA', 'Rank 4 LoRA', 'Rank 8 LoRA', 'Rank 20 LoRA'],
    'Trainable Parameters': [665858, 665858, 665858, 739586, 960770],
    'Total Parameters': [67620868, 67620868, 67620868, 67694596, 67915780],
    'Trainable Parameters %': [0.9846930684178736, 0.9846930684178736, 0.9846930684178736, 1.0925332946813067, 1.4146491433949517],
    'Evaluation Loss': [0.16478970646858215, None, 0.0031875313725322485, 0.002690572990104556, 0.003640302922576666],
    'Evaluation Accuracy': [0.9525, None, 0.999375, 0.9995, 0.99925],
    'Evaluation Runtime': [59.7988, None, 71.0671, 72.4696, 73.5463],
    'Training Loss': [0.29259954833984375, 0.3316435546875, 0.02116710090637207, 0.020570178985595704, 0.023881811141967774],
    'Training Runtime': [124.3835, 468.7667, 223.6335, 226.4168, 228.5311]
}

df = pd.DataFrame(data)


df['Model Size (Bytes)'] = df['Total Parameters'] * 4

df

,Model Variant,Trainable Parameters,Total Parameters,Trainable Parameters %,Evaluation Loss,Evaluation Accuracy,Evaluation Runtime,Training Loss,Training Runtime,Model Size (Bytes)
0,Without LoRA,665858,67620868,0.984693,0.164790,0.952500,59.7988,0.292600,124.3835,270483472
1,QLoRA,665858,67620868,0.984693,NaN,NaN,NaN,0.331644,468.7667,270483472
2,Rank 4 LoRA,665858,67620868,0.984693,0.003188,0.999375,71.0671,0.021167,223.6335,270483472
3,Rank 8 LoRA,739586,67694596,1.092533,0.002691,0.999500,72.4696,0.020570,226.4168,270778384
4,Rank 20 LoRA,960770,67915780,1.414649,0.003640,0.999250,73.5463,0.023882,228.5311,271663120


In [1]:
from datasets import load_dataset, DatasetDict, Dataset

In [3]:
!pip install evaluate

  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl.metadata (18 kB)
  Using cached xxhash-3.4.1-cp38-cp38-win_amd64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py38-none-any.whl.metadata (7.1 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached aiohttp-3.9.3-cp38-cp38-win_amd64.whl.metadata (7.6 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp38-cp38-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp38-cp38-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp38-cp38-win_amd64.whl.metadata (32 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   ---------------------------------------- 84.1/84.1 kB 2.3 MB/s eta 0:00:00
Using cached datasets-2.18.0

In [2]:
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

In [3]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [15]:
model_checkpoint = 'LiyaT3/sentiment-analysis-imdb-distilbert'

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)
for name, param in model.named_parameters():
    param.requires_grad = False
# num=2
# model.layers[-1].trainable= True
# model.layers[-1].units= num

In [16]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [17]:
for name, param in model.named_parameters():
     if name.startswith("classifier"): # choose whatever you like here
        param.requires_grad = True

In [24]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_autoset_attn_implementation',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_check_and_enable_flash_attn_2',
 '_check_and_enable_sdpa',
 '_compiled_call_impl',
 '_convert_head_mask_to_5d',
 '_copy_lm_head_original_to_resized',
 '_create_repo',
 '_dispatch_accelerate_model',
 '_expand_inputs_for_generation',
 '_extract_past_from_model_output',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_k

In [18]:
from datasets import load_dataset

dataset = load_dataset("sg247/binary-classification")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8004
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 2000
    })
})


In [19]:
import pandas as pd

df = pd.DataFrame(dataset['train'])
df.dropna(inplace = True)


df['label'] = df['label'].astype(int)
print(df)


                                                  tweet  label
0     Want to say a huge thanks to @WarriorAssaultS ...      1
1     @jaynehh_ you just need a job and get a letter...      1
2             @knhillrocks HA yes, make it quick tho :D      1
3     @shartyboy Thanks for texting me back :)) I'm ...      1
4     Laying out a greetings card range for print to...      1
...                                                 ...    ...
7999  "@aula_jr: @Jude_Mugabi I'm watching it. 2-0 M...      0
8000                                  why so sudden :((      0
8001        @21oclock :((( bout to instant transmission      0
8002  @ButtCupboard my feet were cold!!!! and there ...      0
8003  @Kimwoobin89__ oppa............./hug ur arm/ w...      0

[8000 rows x 2 columns]


In [20]:
d = pd.DataFrame(dataset['test'])
d.dropna(inplace = True)
d['label'] = d['label'].astype(int)


In [21]:
from datasets import Dataset, DatasetDict
train_dict = {
    'tweet': df['tweet'].tolist(),
    'label': df['label'].tolist()
}
train_dataset = Dataset.from_dict(train_dict)
dataset_dict = DatasetDict({'train': train_dataset})
print(dataset_dict['train']['label'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [22]:
from datasets import Dataset, DatasetDict
test_dict = {
    'tweet': d['tweet'].tolist(),
    'label': d['label'].tolist()
}
test_dataset = Dataset.from_dict(train_dict)
dataset_dict = DatasetDict({'test': train_dataset})

# print(dataset_dict['train']['label'][2])
print(dataset_dict['test']['label'][2])
print(dataset_dict)

1
DatasetDict({
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8000
    })
})


In [23]:
test_dict = {
    'tweet': d['tweet'].tolist(),
    'label': d['label'].tolist()
}

train_dict = {
    'tweet': df['tweet'].tolist(),
    'label': df['label'].tolist()
}



import datasets
train_dataset = datasets.Dataset.from_dict(train_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)
my_dataset_dict = datasets.DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})
print(my_dataset_dict)
print(my_dataset_dict['test']['label'][2])
print(my_dataset_dict['train']['label'][2])

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 2000
    })
})
1
1


In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

def tokenize_function(examples):
    tweets = examples["tweet"]
    preprocess_t= [tweet if isinstance(tweet , str) else str(tweet) for tweet in tweets]
    return tokenizer(preprocess_t , truncation= True , padding= "max_length")
    

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

tokenized_dataset = my_dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [25]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# training 


peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin','k_lin'])

peft_config

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 665,858 || all params: 67,620,868 || trainable%: 0.9846930684178736


In [26]:
lr = 0.01
batch_size = 32
num_epochs = 5

In [35]:
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [36]:
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    eval_dataset = tokenized_dataset['train'],
    compute_metrics=compute_metrics, 
)

In [38]:
trainer.train()

model.to(torch.device('cuda'))



# print("Trained model predictions:")
# print("--------------------------")
# for text in text_list:
#     inputs = tokenizer.encode(text, return_tensors="pt").to(torch.device('cuda'))

#     logits = model(inputs).logits
#     predictions = torch.max(logits,1).indices

#     print(text + " - " + id2label[predictions.tolist()[0]])

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.568759,0.709000


Trained model predictions:
--------------------------


NameError: name 'text_list' is not defined

In [39]:
trainer.save_model("Trained_Model_LoRA")

In [40]:
from transformers import AutoModelForSequenceClassification

path_to_saved_model = "Trained_Model_LoRA"
loaded_model = AutoModelForSequenceClassification.from_pretrained(path_to_saved_model)


In [48]:
inputs = tokenizer.encode("You are ugly", return_tensors="pt")
logits = loaded_model(inputs).logits
predictions = torch.max(logits,1).indices
print(predictions)
print("you are ugly" + " - " + id2label[predictions.tolist()[0]])

tensor([0])
you are ugly - Negative
